# 读取数据集

biored读取

In [1]:
from datasets import load_dataset
datas = load_dataset("bigbio/biored")


/Users/harrychen/Projects/text2knowledge/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
len(datas["test"])

100

biored实体类别

In [3]:
set([j['semantic_type_id'] for i in datas["train"] for j in i['entities']])

{'CellLine',
 'ChemicalEntity',
 'DiseaseOrPhenotypicFeature',
 'GeneOrGeneProduct',
 'OrganismTaxon',
 'SequenceVariant'}

In [8]:
[j for i in datas["train"] for j in i['entities'] if j['semantic_type_id']=='OrganismTaxon']

[{'text': ['patients'],
  'offsets': [[754, 762]],
  'concept_id': '9606',
  'semantic_type_id': 'OrganismTaxon'},
 {'text': ['patients'],
  'offsets': [[1216, 1224]],
  'concept_id': '9606',
  'semantic_type_id': 'OrganismTaxon'},
 {'text': ['patients'],
  'offsets': [[1460, 1468]],
  'concept_id': '9606',
  'semantic_type_id': 'OrganismTaxon'},
 {'text': ['woman'],
  'offsets': [[667, 672]],
  'concept_id': '9606',
  'semantic_type_id': 'OrganismTaxon'},
 {'text': ['patients'],
  'offsets': [[678, 686]],
  'concept_id': '9606',
  'semantic_type_id': 'OrganismTaxon'},
 {'text': ['patients'],
  'offsets': [[827, 835]],
  'concept_id': '9606',
  'semantic_type_id': 'OrganismTaxon'},
 {'text': ['human'],
  'offsets': [[207, 212]],
  'concept_id': '9606',
  'semantic_type_id': 'OrganismTaxon'},
 {'text': ['human'],
  'offsets': [[430, 435]],
  'concept_id': '9606',
  'semantic_type_id': 'OrganismTaxon'},
 {'text': ['patients'],
  'offsets': [[392, 400]],
  'concept_id': '9606',
  'semanti

biored关系类别

In [17]:
set([tri['type'] for d in datas['train'] for tri in d['relations']])

{'Association',
 'Bind',
 'Comparison',
 'Conversion',
 'Cotreatment',
 'Drug_Interaction',
 'Negative_Correlation',
 'Positive_Correlation'}

抽取实体

In [ ]:
import json
import os, sys
import contextlib
from text2knowledge.strategy1 import extract_concepts
from text2knowledge.type_definition import BIORED_ENTITY_TYPES


model_name = 'qwen2.5-7b-instruct-1m'
# base_url = 'http://100.64.0.70:1234/v1'
base_url = 'http://192.168.31.58:1234/v1'

i = 4
while i < len(datas['test']):
    data = datas['test'][i]
    pmid = data['pmid']
    passages = data['passages']
    entities = data['entities']
    abstract = passages[1]['text'][0]
    abstract_offset = passages[1]['offsets'][0]
    metadata = {
        'pmid': pmid,
        'type': 'abstract',
    }

    #with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
    entities_extracted = extract_concepts(abstract, model=model_name, metadata=metadata, base_url=base_url)
    
    if entities_extracted:
        disease_entities_extracted = [e for e in entities_extracted if e['category'] == 'Disease']
        # print(disease_entities_extracted)
        entities_str = json.dumps(entities_extracted, indent=4)
        # print(entities_str)
    else:
        print("No entities found.")
    break

In [5]:
entities_extracted

[{'entity': 'Deoxyguanosine kinase deficiency',
  'confidence': 5,
  'category': 'Disease',
  'pmid': '19394258',
  'type': 'abstract'},
 {'entity': 'DGUOK gene',
  'confidence': 5,
  'category': 'Gene',
  'pmid': '19394258',
  'type': 'abstract'},
 {'entity': 'dGK',
  'confidence': 5,
  'category': 'Protein',
  'pmid': '19394258',
  'type': 'abstract'},
 {'entity': 'mitochondrial DNA depletion',
  'confidence': 5,
  'category': 'Disease',
  'pmid': '19394258',
  'type': 'abstract'},
 {'entity': 'hepatocerebral phenotype',
  'confidence': 4,
  'category': 'Anatomy',
  'pmid': '19394258',
  'type': 'abstract'},
 {'entity': 'DGUOK homozygous mutation (c.444-62C>A)',
  'confidence': 5,
  'category': 'MolecularFunction',
  'pmid': '19394258',
  'type': 'abstract'},
 {'entity': 'cryptic splice site',
  'confidence': 4,
  'category': 'Anatomy',
  'pmid': '19394258',
  'type': 'abstract'},
 {'entity': 'dGK kinase domain',
  'confidence': 5,
  'category': 'Protein',
  'pmid': '19394258',
  'ty

抽取关系

In [2]:
import json
import os, sys
import contextlib
from text2knowledge.strategy1 import graph_prompt
from text2knowledge.type_definition import BIORED_ENTITY_TYPES, BIORED_RELATION_TYPES

model_name = 'qwen2.5-7b-instruct-1m'
# base_url = 'http://100.64.0.70:1234/v1'
base_url = 'http://192.168.31.58:1234/v1'

i = 0
while i < len(datas['test']):
    data = datas['test'][i]
    pmid = data['pmid']
    passages = data['passages']
    relations = data['relations']
    abstract = passages[1]['text'][0]
    abstract_offset = passages[1]['offsets'][0]
    metadata = {
        'pmid': pmid,
        'type': 'abstract',
    }

    # with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
    relations_extracted = graph_prompt(
        abstract,
        model=model_name,
        metadata=metadata,
        base_url=base_url,
        entity_types=BIORED_ENTITY_TYPES,
        relation_types=BIORED_RELATION_TYPES
    )
    
    if relations_extracted:
        # disease_entities_extracted = [e for e in entities_extracted if e['category'] == 'Disease']
        # print(disease_entities_extracted)
        entities_str = json.dumps(relations_extracted, indent=4)
        save_path = f'./extracted/biored/test_{i}.json'
        with open(save_path, 'w') as f:
            f.write(entities_str)
        print(f"Entities saved to {save_path}")
        # print(entities_str)
    else:
        print("No entities found.")
    # break
    i += 1
    # if i >= 6:
    #     break

Entities saved to ./extracted/biored/test_0.json
Entities saved to ./extracted/biored/test_1.json
Entities saved to ./extracted/biored/test_2.json
Entities saved to ./extracted/biored/test_3.json


KeyboardInterrupt: 

In [8]:
relations_extracted

[{'source_name': 'Autism spectrum disorders (ASDs)',
  'source_type': 'Disease',
  'target_name': 'increased rates of anxiety',
  'target_type': 'Condition',
  'relation_type': 'BioMedGPS::AssociatedWith::Disease:Symptom',
  'key_sentence': 'Autism spectrum disorders (ASDs) are heterogeneous disorders presenting with increased rates of anxiety.',
  'pmid': '19565319',
  'type': 'abstract'},
 {'source_name': 'ADORA2A',
  'source_type': 'Gene',
  'target_name': 'panic disorder',
  'target_type': 'Disease',
  'relation_type': 'BioMedGPS::AssociatedWith::Gene:Disease',
  'key_sentence': 'The adenosine A(2A) receptor gene (ADORA2A) is associated with panic disorder.',
  'pmid': '19565319',
  'type': 'abstract'},
 {'source_name': 'ADORA2A',
  'source_type': 'Gene',
  'target_name': 'chromosome 22q11.23',
  'target_type': 'Anatomy',
  'relation_type': 'BioMedGPS::LocatedIn::Gene:Anatomy',
  'key_sentence': 'The adenosine A(2A) receptor gene (ADORA2A) is located on chromosome 22q11.23.',
  'pm